## 1. Grupo 5:

- Jorge Román Butrón
- Fabián Salfate
- Miguel Gil
- Ramón Cornejo



## 1. Dependencias
Instala o actualiza las librer?as necesarias para la tarea.


In [1]:
%pip install -q langchain langchain-openai langchain-community python-dotenv redis



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2. Configuración de entorno
Lee tus credenciales desde `.env` o las variables de entorno antes de usar la API y Redis.


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
REDIS_URL = os.getenv('REDIS_URL')

if not OPENAI_API_KEY:
    raise RuntimeError('Falta OPENAI_API_KEY en .env o variables de entorno.')
if not REDIS_URL:
    raise RuntimeError('Falta REDIS_URL en .env o variables de entorno.')

print('Entorno cargado correctamente.')

Entorno cargado correctamente.


## 3. Manejo de historial en Redis
Funciones para crear, limpiar y revisar el historial de cada persona.


In [ ]:
from typing import Literal
from langchain_community.chat_message_histories import RedisChatMessageHistory

MessageRole = Literal['user', 'ai']
DEFAULT_TTL_SECONDS = 60 * 60 * 24  # 24 horas

def normalize_session_id(raw_name: str) -> str:
    normalized = raw_name.strip().lower()
    if not normalized:
        raise ValueError('El nombre de usuario no puede estar vacío.')
    return normalized

def get_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(
        session_id=normalize_session_id(session_id),
        url=REDIS_URL,
        ttl=DEFAULT_TTL_SECONDS,
    )

def clear_history(session_id: str) -> None:
    get_history(session_id).clear()

def print_history(session_id: str) -> None:
    history = get_history(session_id)
    print(f"Historial de {normalize_session_id(session_id)}:")
    if not history.messages:
        print('  (sin registros)')
        return
    for msg in history.messages:
        speaker = 'Usuario' if msg.type == 'human' else 'Asistente'
        print(f'- {speaker}: {msg.content}')


## 4. Prompt y modelo con salida estructurada


In [ ]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import PydanticOutputParser

system_text = (
    "Eres un asistente hogareño amigable. Recuerda el contexto previo del usuario y responde breve. "
)
class AssistantReply(BaseModel):
    respuesta: str = Field(description='Respuesta principal al usuario.')
    proxima_accion: str = Field(description='Pregunta corta o acción sugerida para continuar la charla.')

parser = PydanticOutputParser(pydantic_object=AssistantReply)
format_instructions = parser.get_format_instructions()

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.4)

prompt = ChatPromptTemplate.from_messages([
    ('system', system_text),
    MessagesPlaceholder(variable_name='history'),
    ('user', '{input}'),
    ('system', 'Responde usando este formato:\n{format_instructions}'),
])

prompt = prompt.partial(format_instructions=format_instructions)

chain_no_parser = prompt | llm


## 5. Entrada tipo `Nombre: mensaje` y helpers

In [5]:
def parse_named_input(raw_text: str) -> tuple[str, str]:
    cleaned = raw_text.strip().strip('"').strip("'")
    cleaned = cleaned.replace('：', ':')
    if ':' not in cleaned:
        raise ValueError("Usa el formato 'Nombre: mensaje'.")
    name, message = cleaned.split(':', 1)
    name = name.strip()
    message = message.strip()
    if not name:
        raise ValueError('Incluye un nombre antes de los dos puntos.')
    if not message:
        raise ValueError('El mensaje no puede estar vacío.')
    return name, message

def ask_assistant(raw_text: str) -> AssistantReply:
    display_name, message = parse_named_input(raw_text)
    session_id = normalize_session_id(display_name)
    history = get_history(session_id)
    user_message = f"{display_name}: {message}"
    ai_msg = chain_no_parser.invoke(
        {
            'history': history.messages,
            'input': user_message,
        }
    )
    history.add_user_message(user_message)
    history.add_ai_message(ai_msg.content)
    return parser.parse(ai_msg.content)

def chat_once(raw_text: str) -> None:
    try:
        display_name, message = parse_named_input(raw_text)
        reply = ask_assistant(raw_text)
    except ValueError as exc:
        print(f"[ALERTA] Entrada inválida: {exc}")
        return
    except KeyError:
        print('[ALERTA] Faltan datos de configuración. Reejecuta las celdas 3 a 5 y reinicia el chat.')
        return
    print(f"{display_name}: {message}")
    print(f"Asistente ({display_name}): {reply.respuesta}")
    print(f"Sugerencia: {reply.proxima_accion}")

## 6. Loop interactivo opcional
Ejecuta esta celda cuando quieras conversar escribiendo `Nombre: mensaje`. Escribe `salir` para terminar.


In [7]:
def chat_loop() -> None:
    print('Escribe tus mensajes como "Nombre: texto". Ejemplo: Fabian: Hola, cómo estás?')
    print('Escribe "salir" para terminar.\n')
    while True:
        raw = input('Tu mensaje: ').strip()
        if not raw:
            continue
        if raw.lower() == 'salir':
            print('Fin del chat interactivo.')
            break
        chat_once(raw)

# Ejecuta chat_loop() cuando quieras iniciar la conversaci?n en vivo:
 #chat_loop()


### Ejecutar el loop interactivo
Corre esta celda solo cuando quieras comenzar a escribir mensajes (`Nombre: texto`).


In [ ]:
# Al ejecutar esta celda iniciar?s el chat en la consola.
chat_loop()


Escribe tus mensajes como "Nombre: texto". Ejemplo: Fabian: Hola, cómo estás?
Escribe "salir" para terminar.

Fabian: Hola, que es un LLM?
Asistente (Fabian): Un LLM, o modelo de lenguaje grande, es un tipo de inteligencia artificial diseñado para entender y generar texto de manera coherente y contextual.
Sugerencia: ¿Te gustaría saber más sobre cómo funcionan los LLM?
Fabian: Que es un auto?
Asistente (Fabian): Un auto es un vehículo motorizado con ruedas, diseñado principalmente para el transporte de personas y mercancías.
Sugerencia: ¿Te gustaría saber sobre los diferentes tipos de autos?
Jorge: Clima de Santiago
Asistente (Jorge): No tengo información actual sobre el clima de Santiago. Te recomiendo consultar un sitio web de meteorología.
Sugerencia: ¿Te gustaría saber algo más sobre Santiago?
Fabian: Que es el sonido?
Asistente (Fabian): El sonido es una vibración que viaja a través de un medio, como el aire o el agua, y que puede ser percibida por el oído humano.
Sugerencia: ¿Qui

## 7. Consulta o limpieza de historiales
Utiliza estas funciones para revisar o reiniciar la conversación de cualquier persona.


In [ ]:
#clear_history('Fabian')
#clear_history('Jorge')
#clear_history('Cleo')

In [18]:

print_history('Fabian')

Historial de fabian:
- Usuario: Fabian: Hola, que es un LLM?
- Asistente: {"respuesta":"Un LLM, o modelo de lenguaje grande, es un tipo de inteligencia artificial diseñado para entender y generar texto de manera coherente y contextual.","proxima_accion":"¿Te gustaría saber más sobre cómo funcionan los LLM?"}
- Usuario: Fabian: Que es un auto?
- Asistente: {"respuesta":"Un auto es un vehículo motorizado con ruedas, diseñado principalmente para el transporte de personas y mercancías.","proxima_accion":"¿Te gustaría saber sobre los diferentes tipos de autos?"}
- Usuario: Fabian: Que es el sonido?
- Asistente: {"respuesta":"El sonido es una vibración que viaja a través de un medio, como el aire o el agua, y que puede ser percibida por el oído humano.","proxima_accion":"¿Quieres saber más sobre cómo se produce el sonido?"}
- Usuario: Fabian: Resumen de lo que estuvimos hablando.
- Asistente: {"respuesta":"Hemos hablado sobre qué es un LLM (modelo de lenguaje grande), qué es un auto (vehícul

In [19]:
print_history('Jorge')

Historial de jorge:
- Usuario: Jorge: Clima de Santiago
- Asistente: {"respuesta":"No tengo información actual sobre el clima de Santiago. Te recomiendo consultar un sitio web de meteorología.","proxima_accion":"¿Te gustaría saber algo más sobre Santiago?"}
- Usuario: Jorge: Describe el pais de Brasil
- Asistente: {"respuesta":"Brasil es el país más grande de América del Sur, conocido por su diversidad cultural, su selva amazónica y sus playas. Tiene una rica historia y es famoso por su música, como la samba y la bossa nova.","proxima_accion":"¿Te gustaría saber más sobre algún aspecto específico de Brasil?"}
- Usuario: Jorge: Resumen de lo que estuvimos hablando.
- Asistente: {"respuesta":"Hablamos sobre el clima de Santiago y te sugerí consultar un sitio de meteorología. También te di un resumen sobre Brasil, destacando su tamaño, diversidad cultural y famosa música.","proxima_accion":"¿Te gustaría profundizar en alguno de estos temas?"}
- Usuario: Jorge: Resumen de lo que estuvimos 

In [20]:
print_history('Cleo')

Historial de cleo:
- Usuario: Cleo: Que es un Iphone?
- Asistente: {"respuesta":"Un iPhone es un teléfono inteligente diseñado y comercializado por Apple. Combina funciones de teléfono, internet y multimedia en un solo dispositivo.","proxima_accion":"¿Te gustaría saber más sobre sus características?"}
- Usuario: Cleo: Resumen de lo que estuvimos hablando.
- Asistente: {"respuesta":"Hablamos sobre qué es un iPhone, que es un teléfono inteligente de Apple que combina funciones de teléfono, internet y multimedia.","proxima_accion":"¿Te gustaría conocer más sobre los modelos de iPhone?"}
- Usuario: Cleo: Resumen de lo que estuvimos hablando.
- Asistente: {"respuesta":"Hablamos sobre qué es un iPhone, un teléfono inteligente de Apple que combina funciones de teléfono, internet y multimedia.","proxima_accion":"¿Te gustaría saber más sobre sus características o modelos?"}
